In [0]:
import pandas as pd
import numpy as np

In [23]:
def extract_article(link):
  l = []
  for s in link.split("/"):
    l.append(s)
  a = l[-2]
  a=a.replace("_"," ")
  return a
extract_article("https://www.reddit.com/r/india/comments/g7qeec/doland_dont_do_press_conference/")

'doland dont do press conference'

In [24]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-04-26 11:48:51--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.141.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.141.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.7MB/s    in 97s     

2020-04-26 11:50:29 (16.1 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [25]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [26]:
vec_rep=pd.read_csv("vector_representation3.csv")
w2v_headline = vec_rep.values.tolist()
w2v_headline = np.array(w2v_headline)
w2v_headline

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.01955003,  0.02229192,  0.03703425, ...,  0.0107328 ,
         0.0176063 ,  0.03468675],
       ...,
       [-0.09240723, -0.04692383,  0.05039062, ...,  0.04433594,
         0.0010498 ,  0.06169434],
       [-0.12124634,  0.08360291,  0.1131897 , ..., -0.01593399,
        -0.08050537,  0.06219482],
       [-0.0425851 , -0.00156948, -0.05125209, ...,  0.05259487,
         0.02280971, -0.01771763]])

In [27]:
def detect_vec(link):
  num_rep = []
  article = extract_article(link)
  print(article)
  vocabulary = list(word2vec.vocab)
  w2Vec_word = np.zeros(300,dtype="float32")
  for word in article.split(" "):
    if word  in  vocabulary:
      w2Vec_word = np.add(w2Vec_word, word2vec[word])
  w2Vec_word = np.divide(w2Vec_word, len(article.split()))
  num_rep.append(w2Vec_word)
  num_rep = np.array(num_rep)
  return num_rep


detect_vec("https://www.reddit.com/r/india/comments/g7qeec/doland_dont_do_press_conference/")

doland dont do press conference


array([[ 1.19140623e-02,  5.35888672e-02,  3.76708992e-02,
         2.78320312e-02, -3.15429680e-02, -4.14062515e-02,
         6.57226592e-02, -9.25292969e-02,  5.58593757e-02,
         8.95996112e-03, -1.25878900e-01, -4.97863777e-02,
        -6.81640655e-02,  5.65673821e-02, -7.99377412e-02,
         4.56054695e-02,  8.75488296e-02,  1.02050781e-01,
         9.68933105e-03, -5.72509766e-02, -2.09960938e-02,
        -1.50588993e-02,  4.73144539e-02,  5.41503914e-02,
         4.76562493e-02,  6.98425323e-02, -4.76379395e-02,
         7.93609619e-02, -4.24316414e-02, -2.25341804e-02,
        -5.12695289e-04,  5.93994148e-02, -5.31005859e-02,
        -1.27587885e-01,  6.36550933e-02,  5.10131828e-02,
         4.39392105e-02,  1.05810545e-01,  6.93847686e-02,
         1.01171874e-01,  6.50451630e-02, -6.60644546e-02,
         1.57055661e-01,  4.58007827e-02, -2.92968750e-02,
        -7.25341812e-02,  1.22070312e-04, -4.44458015e-02,
        -1.06250003e-01,  1.08496092e-01,  6.48437515e-0

In [0]:
 #Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances

In [0]:
import pandas as pd
df = pd.read_csv("data1.csv")

In [0]:
def avg_w2v_based_model(link, num_similar_items=11):
    w2v = detect_vec(link)
    print(w2v[0].reshape(1,-1).shape)
    couple_dist = pairwise_distances(w2v_headline, w2v[0].reshape(1,-1))
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    df0 = pd.DataFrame({'label': df['lebel'][indices].values,
               'headline':df['news'][indices].values,
                'Euclidean similarity with the queried article': couple_dist[indices].ravel()})
    print("="*30,"Queried article details","="*30)
    print('headline : ',df['news'][indices[0]])
    print("\n","="*25,"Recommended articles : ","="*23)
    #return df.iloc[1:,1]
    return df0.iloc[1:,]

In [0]:
def flair_detector(link):
  x= avg_w2v_based_model(link)
  d = {};
  flair =  ""
  for i in x["label"]:
    if i in d.keys():
      
      d[i]=d[i]+1
      
    else:
      d[i]=1;
  mx=0
  for i in x["label"]:
    if d[i]>mx:
      flair = i
      mx =  d[i]
   
  return flair

In [40]:
flair_detector(str("https://www.reddit.com/r/india/comments/g7nuky/armenian_genocide_memorial_at_the_churchyard_of/"))

armenian genocide memorial at the churchyard of
(1, 300)
============================== Queried article details ==============================
headline :  In documentary ‘Scratches on Stone’, erasure and remembrance of Nagaland’s history

 ========================= Recommended articles :  =======================


'IndianCinema'

In [32]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok


In [47]:
from flask  import Flask,render_template,redirect,request


app = Flask(__name__)
run_with_ngrok(app) 

@app.route('/')
def hello():
    return render_template("index.html",my_friends =  friends)

@app.route('/about')
def about():
    return "<h1>about page</h2>"
@app.route('/home')
def home():
    return redirect('/')

titles = []
@app.route('/submit',methods = ["POST"])
def submit_data():
    title = "sa"
    if request.method == "POST":
        link = request.form["link"]
        #page = requests.get(link)
        #soup = BeautifulSoup(page.content,'html.parser')
       # find_all= soup.find_all('h1' ,{"class":"_eYtD2XCVieq6emjKBH3m"})   #extracting title of post 
        flair = flair_detector(link)
        
    return render_template("index.html",flair_is=flair)
       
    

if __name__=='__main__':
    
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://34a66ce9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Apr/2020 12:16:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2020 12:16:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2020 12:17:03] "POST /submit HTTP/1.1" 200 -


doland dont do press conference
(1, 300)
============================== Queried article details ==============================
headline :  LaravelLive India, A Laravel Conference, join us for a conference on fastest growing php framework. Visit laravellive.in for more info

 ========================= Recommended articles :  =======================
